In [2]:
import pickle
import json
import torch

def extract_model_details(file_path):
    # Load the state dictionary from the pickle file
    with open(file_path, 'rb') as f:
        state_dict = pickle.load(f)
    
    # Initialize a dictionary for the model structure
    model_details = {
        "input_layer": {},
        "hidden_layers": [],
        "output_layer": {}
    }

    for layer_name, params in state_dict.items():
        # Parse the layer name (e.g., "fc1.weight" or "fc2.bias")
        if "fc1" in layer_name:
            layer_type = "hidden_layers"
            layer_index = 0  # First hidden layer
        elif "fc2" in layer_name:
            layer_type = "output_layer"
            layer_index = None  # No index for output layer
        else:
            continue  # Skip unknown layers

        if "weight" in layer_name:
            num_nodes = params.shape[0]  # Number of output nodes
            input_dim = params.shape[1]  # Number of input features/nodes

            if layer_type == "hidden_layers":
                # Append details for the hidden layer
                if layer_index == len(model_details["hidden_layers"]):  # Ensure we append
                    model_details["hidden_layers"].append({
                        "weights": params.tolist(),
                        "bias": None,  # Placeholder for bias
                        "num_nodes": num_nodes,
                        "input_dim": input_dim
                    })
            elif layer_type == "output_layer":
                model_details[layer_type] = {
                    "weights": params.tolist(),
                    "bias": None,  # Placeholder for bias
                    "num_nodes": num_nodes,
                    "input_dim": input_dim
                }

        elif "bias" in layer_name:
            if layer_type == "hidden_layers":
                # Add bias to the appropriate hidden layer
                model_details["hidden_layers"][layer_index]["bias"] = params.tolist()
            elif layer_type == "output_layer":
                model_details[layer_type]["bias"] = params.tolist()

    # Add input layer information
    if model_details["hidden_layers"]:
        model_details["input_layer"]["num_nodes"] = model_details["hidden_layers"][0]["input_dim"]
        model_details["input_layer"]["output_nodes"] = model_details["hidden_layers"][0]["num_nodes"]

    return model_details

# Example usage
if __name__ == "__main__":
    model_details = extract_model_details("model.pkl")
    # with open('model_output.json', 'w') as json_file:
    #     json.dump(data, json_file, indent=4)

    # Print the structured dictionary
    print("Model Details:\n")
    print(f"Input Layer: {model_details['input_layer']}")
    print("\nHidden Layers:")
    for i, hidden_layer in enumerate(model_details["hidden_layers"]):
        print(f"  Hidden Layer {i + 1}:")
        for key, value in hidden_layer.items():
            if key == "weights":
                print(f"    {key}: {value[:2]}...")  # Show first 2 rows for brevity
            else:
                print(f"    {key}: {value}")
    print("\nOutput Layer:")
    for key, value in model_details["output_layer"].items():
        if key == "weights":
            print(f"  {key}: {value[:2]}...")  # Show first 2 rows for brevity
        else:
            print(f"  {key}: {value}")


Model Details:

Input Layer: {'num_nodes': 784, 'output_nodes': 128}

Hidden Layers:
  Hidden Layer 1:
    weights: [[-0.027723316103219986, 0.03240501135587692, 0.02736024186015129, -0.006829994730651379, -0.019160646945238113, 0.029106449335813522, 0.008318060077726841, 0.008020423352718353, -0.0011195553233847022, -0.019991345703601837, 0.0323396697640419, 0.013738837093114853, 0.021732602268457413, -0.018649941310286522, 0.027557171881198883, 0.02492823824286461, 0.0025349759962409735, -0.019363921135663986, -0.008887607604265213, 0.029571302235126495, -0.015981893986463547, 0.030207175761461258, -0.015289118513464928, 0.012711227871477604, -0.018431149423122406, 0.0025002446491271257, 0.028014596551656723, 0.03629453480243683, 0.035193562507629395, -0.019141610711812973, 0.019556675106287003, 0.03237681835889816, 0.03382907062768936, -0.025885945186018944, 0.03135392442345619, -0.02152290940284729, 0.030075695365667343, 0.0015418350230902433, -0.025287985801696777, -0.016031645238

In [9]:
import pickle
import json

# Load the state dictionary from the .pkl file
with open('model.pkl', 'rb') as f:
    state_dict = pickle.load(f)

# Function to process the state dictionary and organize layer data
def process_state_dict(state_dict):
    model_data = {}
    layer_data = {}
    layer_index = 0

    for key, value in state_dict.items():
        if "weight" in key:  # Extract weights
            layer_name = f"layer_{layer_index}"
            if layer_name not in layer_data:
                layer_data[layer_name] = {}
            layer_data[layer_name]["weights"] = value.numpy().tolist()
        elif "bias" in key:  # Extract biases
            layer_name = f"layer_{layer_index}"
            if layer_name not in layer_data:
                layer_data[layer_name] = {}
            layer_data[layer_name]["biases"] = value.numpy().tolist()
            layer_index += 1  # Move to the next layer after biases

    # Add activation functions (customize based on your model's architecture)
    for i, layer_name in enumerate(layer_data.keys()):
        if i == 0:
            layer_data[layer_name]["activation"] = "relu"  # Example input layer activation
        elif i == len(layer_data) - 1:
            layer_data[layer_name]["activation"] = "softmax"  # Example output layer activation
        else:
            layer_data[layer_name]["activation"] = "sigmoid"  # Example hidden layer activation

    return layer_data

# Process the state dictionary to extract model data
model_data = process_state_dict(state_dict)

# Save the model data to a JSON file
with open('model.json', 'w') as json_file:
    json.dump(model_data, json_file, indent=4)

print("Model data successfully saved to model.json")


Model data successfully saved to model.json


In [10]:
import json

# Load the model data from the JSON file
with open('model.json', 'r') as json_file:
    model_data = json.load(json_file)

# Print the model data layer by layer
print("Model Data from JSON:")
for layer, details in model_data.items():
    print(f"\n{layer}:")
    print(f"  Weights: {details['weights']}")
    print(f"  Biases: {details['biases']}")
    print(f"  Activation Function: {details['activation']}")


Model Data from JSON:

layer_0:
  Weights: [[-0.027723316103219986, 0.03240501135587692, 0.02736024186015129, -0.006829994730651379, -0.019160646945238113, 0.029106449335813522, 0.008318060077726841, 0.008020423352718353, -0.0011195553233847022, -0.019991345703601837, 0.0323396697640419, 0.013738837093114853, 0.021732602268457413, -0.018649941310286522, 0.027557171881198883, 0.02492823824286461, 0.0025349759962409735, -0.019363921135663986, -0.008887607604265213, 0.029571302235126495, -0.015981893986463547, 0.030207175761461258, -0.015289118513464928, 0.012711227871477604, -0.018431149423122406, 0.0025002446491271257, 0.028014596551656723, 0.03629453480243683, 0.035193562507629395, -0.019141610711812973, 0.019556675106287003, 0.03237681835889816, 0.03382907062768936, -0.025885945186018944, 0.03135392442345619, -0.02152290940284729, 0.030075695365667343, 0.0015418350230902433, -0.025287985801696777, -0.016031645238399506, 0.02885415405035019, 0.01564973220229149, -0.023614760488271713, 